In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Step 1: Import Libraries

In [ ]:
# Import Libraries

import time
import pickle
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from sklearn.svm import SVC
from sklearn.metrics import auc
from collections import Counter
warnings.simplefilter("ignore")
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn import preprocessing
from prettytable import PrettyTable 
from sklearn.metrics import roc_curve
from astropy.table import Table, Column
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

###### SMOTE and RandomUnderSampler (for balancing data) ############

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

from imblearn.over_sampling import SVMSMOTE 
from imblearn.over_sampling import ADASYN 
from imblearn.under_sampling import NearMiss 

# Step 2: Read Dataset

In [ ]:
# Load Dataset

data = pd.read_excel('drive/My Drive/Colab Notebooks/WisconsinDataset/Forms_cleaned.xlsx', 'form1')

print('\n\nDataset:')
print('========\n')
data



Dataset:



,EID,FormID,Flagged,Attempt,tot_time,iresp.1,iresp.2,iresp.3,iresp.4,iresp.5,iresp.6,iresp.7,iresp.8,iresp.9,iresp.10,iresp.11,iresp.12,iresp.13,iresp.14,iresp.15,iresp.16,iresp.17,iresp.18,iresp.19,iresp.20,iresp.21,iresp.22,iresp.23,iresp.24,iresp.25,iresp.26,iresp.27,iresp.28,iresp.29,iresp.30,iresp.31,iresp.32,iresp.33,iresp.34,iresp.35,...,idur.131,idur.132,idur.133,idur.134,idur.135,idur.136,idur.137,idur.138,idur.139,idur.140,idur.141,idur.142,idur.143,idur.144,idur.145,idur.146,idur.147,idur.148,idur.149,idur.150,idur.151,idur.152,idur.153,idur.154,idur.155,idur.156,idur.157,idur.158,idur.159,idur.160,idur.161,idur.162,idur.163,idur.164,idur.165,idur.166,idur.167,idur.168,idur.169,idur.170
0,e100001,form1,0,5,10133,1,1,3.0,1,2,1.0,4,4.0,1,1,3,1.0,2,2.0,4,1.0,2,1.0,4,3,3.0,3.0,4,3,3.0,1.0,2.0,1.0,1.0,2.0,4.0,2,1,2.0,1.0,...,85,47,60,46,98,106,92,63,103,48,42,66,26,57,14,63,45,28,21,15,64,23,61,28,35,28,64,122,20,18,17,102,55,50,17,28,53,82,100,21
1,e100002,form1,1,1,12409,1,3,3.0,1,1,3.0,4,4.0,2,4,3,2.0,1,2.0,3,4.0,2,2.0,3,3,3.0,3.0,3,1,3.0,2.0,4.0,1.0,4.0,4.0,2.0,1,1,4.0,1.0,...,191,48,52,108,69,59,93,133,136,94,139,43,23,82,56,32,77,87,75,34,98,30,107,42,126,31,47,51,83,64,22,121,114,44,36,53,120,93,87,29
2,e100003,form1,1,1,12457,2,3,4.0,4,2,3.0,3,4.0,2,3,1,3.0,4,4.0,4,1.0,2,1.0,3,2,4.0,3.0,4,3,4.0,1.0,2.0,4.0,3.0,3.0,1.0,4,4,1.0,3.0,...,62,54,89,72,68,78,77,77,170,48,84,62,52,122,41,98,38,49,52,27,62,18,78,47,109,47,44,72,23,16,73,34,92,83,50,30,63,96,79,48
3,e100004,form1,1,3,11389,1,1,1.0,4,2,1.0,4,4.0,4,4,3,4.0,4,1.0,4,2.0,2,3.0,3,2,2.0,3.0,3,1,1.0,1.0,2.0,3.0,2.0,2.0,4.0,3,2,4.0,3.0,...,79,69,105,75,60,64,77,54,73,50,60,29,37,72,34,78,74,93,59,39,39,30,34,35,62,33,47,87,83,45,30,93,49,26,57,41,59,41,84,17
4,e100005,form1,1,5,13669,1,1,2.0,4,2,2.0,1,4.0,2,1,4,NaN,2,1.0,4,4.0,1,2.0,3,3,3.0,3.0,3,4,4.0,1.0,NaN,1.0,1.0,NaN,3.0,2,4,4.0,1.0,...,171,29,92,56,55,189,146,108,151,71,57,31,33,138,131,92,110,30,57,20,34,34,72,56,96,46,119,66,133,34,11,0,58,127,63,68,164,74,63,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,e101632,form1,0,1,8760,1,1,3.0,3,2,2.0,3,1.0,4,1,4,4.0,4,4.0,3,1.0,2,1.0,1,1,3.0,3.0,3,4,4.0,3.0,4.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,43,59,61,70,24,67,46,41,86,47,54,24,46,36,68,39,66,33,34,13,38,35,37,23,64,20,19,72,63,15,10,72,40,10,10,52,56,47,61,19
1632,e101633,form1,0,1,10568,1,1,3.0,3,2,2.0,3,1.0,1,1,4,3.0,4,4.0,3,1.0,2,1.0,1,1,3.0,3.0,3,4,4.0,3.0,1.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,62,42,22,26,35,44,69,69,92,79,79,18,17,48,230,58,44,90,60,18,74,49,149,14,28,38,27,128,52,8,11,116,45,11,7,116,79,59,48,40
1633,e101634,form1,0,1,12449,1,1,3.0,3,2,2.0,3,1.0,3,1,4,1.0,4,4.0,3,1.0,3,1.0,1,1,2.0,3.0,3,4,4.0,3.0,1.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,165,31,52,35,27,80,47,59,78,90,137,42,35,35,52,109,75,114,36,33,34,168,62,68,44,18,36,144,98,11,47,164,28,11,30,54,19,27,43,14
1634,e101635,form1,0,1,8209,1,1,3.0,3,2,2.0,3,1.0,4,1,4,3.0,2,4.0,3,1.0,2,1.0,1,1,3.0,3.0,3,4,4.0,3.0,1.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,42,36,12,33,26,57,49,38,123,39,90,13,41,38,17,36,41,30,26,32,50,18,69,45,30,24,27,50,24,43,14,45,32,14,28,17,18,109,94,14


# Step 3: Understand Datset

In [ ]:
# Understand Dataset

print("\n\nAttributes of Dataset:")
print("======================\n")

print("\nName of Dataset: Forms_cleaned")
print("==============================\n")

print("\nNumber of Rows:", len(data))
print("====================\n")

print("\nNumber of Columns:", len(data.columns))
print("======================\n")

print("\nClass Distribution:")
print("===================\n")
print(data['Flagged'].value_counts())



Attributes of Dataset:


Name of Dataset: Forms_cleaned


Number of Rows: 1636


Number of Columns: 515


Class Distribution:

0    1590
1      46
Name: Flagged, dtype: int64


# Step 4: Pre-Process Dataset

In [ ]:
# Pre-Processing

data.tail()
data.drop(data.loc[:, 'iresp.1':'iresp.170'], inplace = True, axis = 1) 
data_final = data

In [ ]:
# Count Null Values

print(data_final.isnull().sum())
print(data_final.shape)

EID         0
FormID      0
Flagged     0
Attempt     0
tot_time    0
           ..
idur.166    0
idur.167    0
idur.168    0
idur.169    0
idur.170    0
Length: 345, dtype: int64
(1636, 345)


In [ ]:
# Time

data_final['mean_idur'] = data_final.loc[:, 'idur.1':'idur.170'].mean(axis=1)
data_final['median_idur'] = data_final.loc[:, 'idur.1':'idur.170'].median(axis=1)
data_final['max_idur'] = data_final.loc[:, 'idur.1':'idur.170'].max(axis=1)
data_final['min_idur'] = data_final.loc[:, 'idur.1':'idur.170'].min(axis=1)

# Correct Answers

data_final['iraw_score'] = data_final.loc[:, 'iraw.1':'iraw.170'].sum(axis=1)
data_final['iraw_score_pct'] = data_final['iraw_score']/170
data_final

,EID,FormID,Flagged,Attempt,tot_time,iraw.1,iraw.2,iraw.3,iraw.4,iraw.5,iraw.6,iraw.7,iraw.8,iraw.9,iraw.10,iraw.11,iraw.12,iraw.13,iraw.14,iraw.15,iraw.16,iraw.17,iraw.18,iraw.19,iraw.20,iraw.21,iraw.22,iraw.23,iraw.24,iraw.25,iraw.26,iraw.27,iraw.28,iraw.29,iraw.30,iraw.31,iraw.32,iraw.33,iraw.34,iraw.35,...,idur.137,idur.138,idur.139,idur.140,idur.141,idur.142,idur.143,idur.144,idur.145,idur.146,idur.147,idur.148,idur.149,idur.150,idur.151,idur.152,idur.153,idur.154,idur.155,idur.156,idur.157,idur.158,idur.159,idur.160,idur.161,idur.162,idur.163,idur.164,idur.165,idur.166,idur.167,idur.168,idur.169,idur.170,mean_idur,median_idur,max_idur,min_idur,iraw_score,iraw_score_pct
0,e100001,form1,0,5,10133,1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,...,92,63,103,48,42,66,26,57,14,63,45,28,21,15,64,23,61,28,35,28,64,122,20,18,17,102,55,50,17,28,53,82,100,21,56.323529,50.0,143,14,54,0.317647
1,e100002,form1,1,1,12409,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,...,93,133,136,94,139,43,23,82,56,32,77,87,75,34,98,30,107,42,126,31,47,51,83,64,22,121,114,44,36,53,120,93,87,29,69.158824,63.0,202,11,55,0.323529
2,e100003,form1,1,1,12457,0,0,0,0,1,0,1,0,0,0,0,1,1,1,0,1,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,...,77,77,170,48,84,62,52,122,41,98,38,49,52,27,62,18,78,47,109,47,44,72,23,16,73,34,92,83,50,30,63,96,79,48,69.547059,65.5,179,16,61,0.358824
3,e100004,form1,1,3,11389,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,1,...,77,54,73,50,60,29,37,72,34,78,74,93,59,39,39,30,34,35,62,33,47,87,83,45,30,93,49,26,57,41,59,41,84,17,62.882353,62.0,154,12,61,0.358824
4,e100005,form1,1,5,13669,1,1,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,1,0,0,1,1,0,1,0,...,146,108,151,71,57,31,33,138,131,92,110,30,57,20,34,34,72,56,96,46,119,66,133,34,11,0,58,127,63,68,164,74,63,17,76.547059,69.0,189,0,62,0.364706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,e101632,form1,0,1,8760,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,...,46,41,86,47,54,24,46,36,68,39,66,33,34,13,38,35,37,23,64,20,19,72,63,15,10,72,40,10,10,52,56,47,61,19,48.658824,43.0,152,8,156,0.917647
1632,e101633,form1,0,1,10568,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,69,69,92,79,79,18,17,48,230,58,44,90,60,18,74,49,149,14,28,38,27,128,52,8,11,116,45,11,7,116,79,59,48,40,59.335294,49.0,230,7,153,0.900000
1633,e101634,form1,0,1,12449,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,47,59,78,90,137,42,35,35,52,109,75,114,36,33,34,168,62,68,44,18,36,144,98,11,47,164,28,11,30,54,19,27,43,14,69.894118,47.0,322,7,154,0.905882
1634,e101635,form1,0,1,8209,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,49,38,123,39,90,13,41,38,17,36,41,30,26,32,50,18,69,45,30,24,27,50,24,43,14,45,32,14,28,17,18,109,94,14,45.376471,34.0,258,10,152,0.894118


# Step 5: Feature Selection

In [ ]:
# iraw_idur_OEF (both+OEF)

col_features = ['Attempt', 'tot_time', 'mean_idur', 'median_idur', 'max_idur', 'min_idur', 'iraw_score_pct']

for x in range(1,171):
    col_features.append('idur.'+str(x))

for i in range(1,171):
    col_features.append('iraw.'+str(i))

X = data_final.loc[:, col_features]
Y = data_final.Flagged    

In [ ]:
# Count Values

counter = Counter(Y)
print(counter)

Counter({0: 1590, 1: 46})


# Step 7: Feature Scaling

In [ ]:
# Import Libraries
from sklearn.preprocessing import MinMaxScaler
num_cols = X.columns

# apply standardization on numerical features
for i in num_cols:
    # fit on training data column
    scale = MinMaxScaler(feature_range=(0, 1)).fit(X[[i]])

    # transform the training data column
    X[i] = scale.transform(X[[i]])
    
print("\n\nTraining Features After Scaling: ")
print("=================================\n")
X



Training Features After Scaling: 



,Attempt,tot_time,mean_idur,median_idur,max_idur,min_idur,iraw_score_pct,idur.1,idur.2,idur.3,idur.4,idur.5,idur.6,idur.7,idur.8,idur.9,idur.10,idur.11,idur.12,idur.13,idur.14,idur.15,idur.16,idur.17,idur.18,idur.19,idur.20,idur.21,idur.22,idur.23,idur.24,idur.25,idur.26,idur.27,idur.28,idur.29,idur.30,idur.31,idur.32,idur.33,...,iraw.131,iraw.132,iraw.133,iraw.134,iraw.135,iraw.136,iraw.137,iraw.138,iraw.139,iraw.140,iraw.141,iraw.142,iraw.143,iraw.144,iraw.145,iraw.146,iraw.147,iraw.148,iraw.149,iraw.150,iraw.151,iraw.152,iraw.153,iraw.154,iraw.155,iraw.156,iraw.157,iraw.158,iraw.159,iraw.160,iraw.161,iraw.162,iraw.163,iraw.164,iraw.165,iraw.166,iraw.167,iraw.168,iraw.169,iraw.170
0,1.0,0.251140,0.254294,0.246154,0.037668,0.482759,0.000000,0.169065,0.041812,0.114007,0.066667,0.057377,0.126761,0.146814,0.323276,0.144886,0.232092,0.100649,0.136253,0.032258,0.181661,0.133820,0.119741,0.076754,0.113065,0.114286,0.077647,0.180812,0.227074,0.040541,0.087935,0.491597,0.380645,0.139319,0.131673,0.270916,0.119741,0.173034,0.305556,0.059809,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.2,0.373250,0.379208,0.379487,0.090583,0.379310,0.009804,0.151079,0.081882,0.250814,0.112121,0.196721,0.161972,0.119114,0.366379,0.051136,0.372493,0.152597,0.131387,0.039427,0.129758,0.216545,0.165049,0.096491,0.218593,0.354286,0.070588,0.154982,0.048035,0.139640,0.077710,0.256303,0.206452,0.123839,0.206406,0.338645,0.116505,0.215730,0.166667,0.028708,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.2,0.375825,0.382986,0.405128,0.069955,0.551724,0.068627,0.201439,0.055749,0.179153,0.042424,0.200820,0.373239,0.077562,0.512931,0.113636,0.146132,0.113636,0.243309,0.093190,0.110727,0.094891,0.271845,0.140351,0.236181,0.297143,0.298824,0.313653,0.144105,0.162162,0.153374,0.474790,0.296774,0.266254,0.459075,0.362550,0.391586,0.242697,0.229167,0.155502,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.6,0.318526,0.318125,0.369231,0.047534,0.413793,0.068627,0.219424,0.247387,0.153094,0.112121,0.262295,0.246479,0.041551,0.228448,0.238636,0.257880,0.074675,0.218978,0.189964,0.171280,0.199513,0.171521,0.028509,0.125628,0.234286,0.171765,0.184502,0.056769,0.056306,0.094070,0.285714,0.212903,0.191950,0.309609,0.155378,0.077670,0.146067,0.166667,0.105263,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
4,1.0,0.440850,0.451111,0.441026,0.078924,0.000000,0.078431,0.406475,0.139373,0.250814,0.178788,0.258197,0.193662,0.152355,0.155172,0.485795,0.303725,0.116883,0.000000,0.462366,0.057093,0.211679,0.239482,0.162281,0.228643,0.914286,0.211765,0.143911,0.183406,0.322072,0.051125,0.285714,0.258065,0.000000,0.252669,0.474104,0.000000,0.076404,0.531250,0.086124,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,0.2,0.177477,0.179700,0.174359,0.045740,0.275862,1.000000,0.118705,0.073171,0.120521,0.339394,0.241803,0.154930,0.052632,0.056034,0.039773,0.203438,0.050325,0.133820,0.050179,0.044983,0.077859,0.090615,0.052632,0.213568,0.171429,0.265882,0.129151,0.034934,0.094595,0.110429,0.138655,0.148387,0.436533,0.081851,0.131474,0.093851,0.051685,0.326389,0.066986,...,1.0,1.0,1.0

# Step 8: Automatic Features Selection

<div class="alert alert-info">
<b>4 Automatic Features Selection Methods:</b>
<div class="alert alert-success">
<b>
    <code>(1)</code> Chi2 Features Selection <br><br>
    <code>(2)</code> ExtraTree Features Selection <br><br>
    <code>(3)</code> REF Features Selection <br><br>
    <code>(4)</code> Pearson Correlation Features Selection <br><br>
</b>
</div>
</div>

## Step 8.1: Chi2 Features Selection

## Step 8.2: Extra Tree Features Selection

<div class="alert alert-warning">
<b>This class implements a meta estimator that fits a number of randomized decision trees (a.k.a. extra-trees) on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.</b>
</div>

In [ ]:
# Import Libraries
# from sklearn.ensemble import ExtraTreesClassifier

# model = ExtraTreesClassifier()
# model.fit(X,Y)
# columns = X.columns.values

# print(model.feature_importances_)
# feat_importances = pd.Series(model.feature_importances_)

In [ ]:
# Print 20 Best Features

# top = feat_importances.nlargest(20)
# feature = top.index
# names = columns[feature]

# top_features = pd.DataFrame()
# top_features["Features"]=names
# top_features["    Score"]=top.values
# top_features["Ranking"] = ["       --->      Rank-01","       --->      Rank-02","       --->      Rank-03","       --->      Rank-04","       --->      Rank-05","       --->      Rank-06","       --->      Rank-07","       --->      Rank-08","       --->      Rank-09","       --->      Rank-10","       --->      Rank-11","       --->      Rank-12","       --->      Rank-13","       --->      Rank-14","       --->      Rank-15","       --->      Rank-16","       --->      Rank-17","       --->      Rank-18","       --->      Rank-19","       --->      Rank-20"]
# print("\n===================================================\n")
# print("================== TOP 10 FEATURES ================")
# print("\n===================================================\n")
# print(top_features.to_string(index=False))
# print("\n===================================================\n")

### Features Set-2

In [ ]:
# Features_Set2 = X[top_features["Features"]]

# print("\n\nAutomatic Selected Features Set 2:")
# print("==================================\n")
# Features_Set2

<center>
    <br>
        <br>
            <br>
                <h5 style = "color:blue">====================================================================</h5>
                <h2 style = "color:green">Training Phase</h2>
                <h5 style = "color:blue">====================================================================</h5>
            <br>
        <br>
    <br>
</center>

# Step 12: Train Models of Features Set-4

## Step 12.1: Train Test Split

In [ ]:
# Train Test Split

#X_train, X_test, y_train, y_test= train_test_split(Features_Set2, Y, test_size= 0.2, random_state=64 ,shuffle = True)

##### top 20 features by 1 selection alg. #########
# X_train, X_test, y_train, y_test= train_test_split(Features_Set2, Y, test_size= 0.2, random_state=23 ,shuffle = True)
# X_train1, X_pred_train, y_train1, y_pred_train= train_test_split(X_train, y_train, test_size= 0.25, random_state=23 ,shuffle = True)

##### X, Y ##########
X_train, X_test, y_train, y_test= train_test_split(X, Y, test_size= 0.2, random_state=23 ,shuffle = True)
X_train1, X_pred_train, y_train1, y_pred_train= train_test_split(X_train, y_train, test_size= 0.25, random_state=23 ,shuffle = True)


##### top features from 4 selection alg. plus summatives, and resampling ##########
### both 33%
#over = SMOTE(sampling_strategy=0.4)
##over = SVMSMOTE(sampling_strategy=0.4, random_state=42, k_neighbors=10)
#over = ADASYN(sampling_strategy=0.4, random_state=42)  ###bad

#under = RandomUnderSampler(sampling_strategy=0.5)
##under = NearMiss(sampling_strategy=1)

##steps = [('o', over), ('u', under)]

##pipeline = Pipeline(steps=steps)

##X_train, X_test, y_train, y_test= train_test_split(X, Y, test_size= 0.2, random_state=23 ,shuffle = True)
##X_train1_orig, X_pred_train, y_train1_orig, y_pred_train= train_test_split(X_train, y_train, test_size= 0.25, random_state=23 ,shuffle = True)

##X_train1, y_train1 = pipeline.fit_resample(X_train1_orig, y_train1_orig)


## Step 12.3: Random Forest Classifier

In [ ]:
# Training

clf1 = RandomForestClassifier(criterion="gini", max_depth=500, max_features="auto", min_samples_leaf=1, min_samples_split=2, n_estimators=911)
clf1.fit(X_train1, y_train1)

RandomForestClassifier(max_depth=500, n_estimators=911)

### Evalute the performace

In [ ]:
# Predictions
y_pred = clf1.predict(X_pred_train)


tn, fp, fn, tp = confusion_matrix(y_pred_train, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_pred_train, y_pred))
print("  recall: ", recall_score(y_pred_train, y_pred))
print("  f1-score: ", f1_score(y_pred_train, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

  precision:  0.0
  recall:  0.0
  f1-score:  0.0
  fpr:  0.0
  tn:  320
  fp:  0
  fn:  7
  tp:  0
  real_cheaters:  7


## Step 12.6: Gradient Boosting Classifier

In [ ]:
# Training

clf2 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100)
clf2.fit(X_train1, y_train1)


GradientBoostingClassifier()

### Evalute the performace

In [ ]:
# Predictions
y_pred = clf2.predict(X_pred_train)

tn, fp, fn, tp = confusion_matrix(y_pred_train, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_pred_train, y_pred))
print("  recall: ", recall_score(y_pred_train, y_pred))
print("  f1-score: ", f1_score(y_pred_train, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

  precision:  0.3333333333333333
  recall:  0.14285714285714285
  f1-score:  0.2
  fpr:  0.00625
  tn:  318
  fp:  2
  fn:  6
  tp:  1
  real_cheaters:  7


In [ ]:
clf3 = MLPClassifier(activation="relu", learning_rate="invscaling", solver="adam")
clf3.fit(X_train1, y_train1)

# Predictions
y_pred = clf3.predict(X_pred_train)

tn, fp, fn, tp = confusion_matrix(y_pred_train, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_pred_train, y_pred))
print("  recall: ", recall_score(y_pred_train, y_pred))
print("  f1-score: ", f1_score(y_pred_train, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

  precision:  0.3333333333333333
  recall:  0.14285714285714285
  f1-score:  0.2
  fpr:  0.00625
  tn:  318
  fp:  2
  fn:  6
  tp:  1
  real_cheaters:  7


In [ ]:
clf4 = GaussianNB(var_smoothing=6.579332246575682e-08)
clf4.fit(X_train1, y_train1)

# Predictions
y_pred = clf4.predict(X_pred_train)

tn, fp, fn, tp = confusion_matrix(y_pred_train, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_pred_train, y_pred))
print("  recall: ", recall_score(y_pred_train, y_pred))
print("  f1-score: ", f1_score(y_pred_train, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

  precision:  0.05555555555555555
  recall:  0.2857142857142857
  f1-score:  0.09302325581395349
  fpr:  0.10625
  tn:  286
  fp:  34
  fn:  5
  tp:  2
  real_cheaters:  7


# Step 13: Combine Models with Stacking

In [ ]:
# Prediction of all models
 #   model2 = StackingClassifier(estimators7, final_estimator=LogisticRegression() )
 #   model2 = StackingClassifier(estimators7, final_estimator=SVC(kernel='linear', probability=True))
 #   model2 = StackingClassifier(estimators7, final_estimator=GradientBoostingClassifier() )
 #   model2 = StackingClassifier(estimators7, final_estimator=RandomForestClassifier() )
  #   model2 = StackingClassifier(estimators7, final_estimator=DecisionTreeClassifier() )

P1 = clf1.predict_proba(X_pred_train)
P1 = P1[:, 1]
P2 = clf2.predict_proba(X_pred_train)
P2 = P2[:, 1]
P3 = clf3.predict_proba(X_pred_train)
P3 = P3[:, 1]
P4 = clf4.predict_proba(X_pred_train)
P4 = P4[:, 1]

X_pred_train["Predict1"] = P1
X_pred_train["Predict2"] = P2
X_pred_train["Predict3"] = P3
X_pred_train["Predict4"] = P4
X_pred_train

,Attempt,tot_time,mean_idur,median_idur,max_idur,min_idur,iraw_score_pct,idur.1,idur.2,idur.3,idur.4,idur.5,idur.6,idur.7,idur.8,idur.9,idur.10,idur.11,idur.12,idur.13,idur.14,idur.15,idur.16,idur.17,idur.18,idur.19,idur.20,idur.21,idur.22,idur.23,idur.24,idur.25,idur.26,idur.27,idur.28,idur.29,idur.30,idur.31,idur.32,idur.33,...,iraw.135,iraw.136,iraw.137,iraw.138,iraw.139,iraw.140,iraw.141,iraw.142,iraw.143,iraw.144,iraw.145,iraw.146,iraw.147,iraw.148,iraw.149,iraw.150,iraw.151,iraw.152,iraw.153,iraw.154,iraw.155,iraw.156,iraw.157,iraw.158,iraw.159,iraw.160,iraw.161,iraw.162,iraw.163,iraw.164,iraw.165,iraw.166,iraw.167,iraw.168,iraw.169,iraw.170,Predict1,Predict2,Predict3,Predict4
1204,0.8,0.211546,0.221204,0.189744,0.066368,0.379310,0.656863,0.187050,0.090592,0.100977,0.069697,0.114754,0.098592,0.074792,0.112069,0.133523,0.077364,0.201299,0.323601,0.311828,0.095156,0.038929,0.122977,0.206140,0.236181,0.062857,0.042353,0.261993,0.056769,0.065315,0.141104,0.537815,0.161290,0.145511,0.128114,0.147410,0.145631,0.083146,0.201389,0.038278,...,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.004391,0.001233,0.000045,4.645779e-15
1222,0.6,0.385858,0.381383,0.328205,0.095964,0.413793,0.686275,0.194245,0.043554,0.114007,0.027273,0.200820,0.123239,0.049861,0.250000,0.119318,0.332378,0.162338,0.435523,0.089606,0.064014,0.133820,0.064725,0.133772,0.160804,0.457143,0.232941,0.136531,0.052402,0.085586,0.057260,0.189076,0.329032,0.352941,0.352313,0.179283,0.148867,0.103371,0.173611,0.133971,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.002195,0.001233,0.000005,1.432019e-27
653,0.2,0.369548,0.359171,0.297436,0.200897,0.275862,0.843137,0.032374,0.026132,0.091205,0.087879,0.155738,0.098592,0.229917,0.133621,0.053977,0.128940,0.301948,0.401460,0.111111,0.117647,0.021898,0.271845,0.072368,0.150754,0.131429,0.070588,0.084871,0.056769,0.146396,0.044990,0.294118,0.174194,0.133127,0.153025,0.278884,0.262136,0.089888,0.114583,0.105263,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.023052,0.001233,0.000014,1.102675e-60
1482,0.2,0.147272,0.145638,0.158974,0.019731,0.275862,0.794118,0.089928,0.121951,0.087948,0.003030,0.143443,0.133803,0.016620,0.146552,0.102273,0.074499,0.113636,0.124088,0.168459,0.072664,0.055961,0.233010,0.052632,0.138191,0.114286,0.256471,0.136531,0.043668,0.015766,0.061350,0.138655,0.167742,0.164087,0.188612,0.326693,0.168285,0.074157,0.079861,0.035885,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.025247,0.001233,0.000304,8.255824e-01
377,0.4,0.384409,0.370678,0.328205,0.327354,0.413793,0.656863,0.323741,0.247387,0.355049,0.121212,0.106557,0.235915,0.074792,0.241379,0.099432,0.111748,0.149351,0.347932,0.114695,0.115917,0.060827,0.220065,0.061404,0.097990,0.085714,0.141176,0.313653,0.056769,0.189189,0.079755,0.214286,0.406452,0.130031,0.174377,0.187251,0.090615,0.094382,0.149306,0.174641,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.018661,0.001233,0.000108,1.994732e-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,0.2,0.378937,0.385562,0.384615,0.086996,0.551724,0.950980,0.258993,0.099303,0.061889,0.121212,0.225410,0.193662,0.058172,0.379310,0.102273,0.275072,0.163961,0.248175,0.222222,0.110727,0.323601,0.161812,0.149123,0.140704,0.194286,0.188235

In [ ]:

# summatives + top features from 4 selection algorithm + prediction1-4
col_features = ['Attempt', 'tot_time', 'mean_idur', 'median_idur', 'max_idur', 'min_idur', 'iraw_score_pct', 'iraw.32', 'iraw.77', 'iraw.147', 'iraw.57', 'iraw.86', 'iraw.152', 'iraw.12', 'iraw.127', 'iraw.140', 'iraw.96', 'idur.146', 'idur.36', 'idur.114', 'idur.84', 'idur.142', 'idur.60', 'idur.71', 'idur.133', 'idur.88', 'idur.59', 'idur.69', 'idur.38', 'idur.67', 'idur.81', 'idur.96', 'idur.162', 'Predict1', 'Predict2', 'Predict3', 'Predict4']
X_pred_train_meta = X_pred_train.loc[:, col_features]


In [ ]:
X_pred_train_meta

,Attempt,tot_time,mean_idur,median_idur,max_idur,min_idur,iraw_score_pct,iraw.32,iraw.77,iraw.147,iraw.57,iraw.86,iraw.152,iraw.12,iraw.127,iraw.140,iraw.96,idur.146,idur.36,idur.114,idur.84,idur.142,idur.60,idur.71,idur.133,idur.88,idur.59,idur.69,idur.38,idur.67,idur.81,idur.96,idur.162,Predict1,Predict2,Predict3,Predict4
1204,0.8,0.211546,0.221204,0.189744,0.066368,0.379310,0.656863,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.156576,0.144828,0.052632,0.060606,0.076190,0.101266,0.087209,0.058228,0.195918,0.063830,0.096579,0.091358,0.200573,0.102094,0.120,0.182561,0.004391,0.001233,0.000045,4.645779e-15
1222,0.6,0.385858,0.381383,0.328205,0.095964,0.413793,0.686275,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.225470,0.120690,0.074013,0.104683,0.092063,0.104430,0.267442,0.154430,0.224490,0.140426,0.092555,0.059259,0.166189,0.465969,0.740,0.332425,0.002195,0.001233,0.000005,1.432019e-27
653,0.2,0.369548,0.359171,0.297436,0.200897,0.275862,0.843137,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.066806,0.213793,0.148849,0.019284,0.107937,0.259494,0.250000,0.040506,0.130612,0.106383,0.088531,0.330864,0.177650,0.065445,0.325,0.234332,0.023052,0.001233,0.000014,1.102675e-60
1482,0.2,0.147272,0.145638,0.158974,0.019731,0.275862,0.794118,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.098121,0.027586,0.062500,0.118457,0.063492,0.060127,0.116279,0.101266,0.097959,0.114894,0.076459,0.071605,0.137536,0.167539,0.245,0.152589,0.025247,0.001233,0.000304,8.255824e-01
377,0.4,0.384409,0.370678,0.328205,0.327354,0.413793,0.656863,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.273486,0.089655,0.073191,0.046832,0.139683,0.177215,0.110465,0.073418,0.126531,0.038298,0.257545,0.162963,0.249284,0.209424,0.175,0.152589,0.018661,0.001233,0.000108,1.994732e-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,0.2,0.378937,0.385562,0.384615,0.086996,0.551724,0.950980,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.102296,0.258621,0.114309,0.104683,0.123810,0.072785,0.098837,0.146835,0.375510,0.161702,0.110664,0.333333,0.223496,0.416230,0.310,0.305177,0.000000,0.001233,0.000010,2.735145e-27
462,0.2,0.396588,0.405370,0.317949,0.117489,0.448276,0.705882,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.235908,0.282759,0.171053,0.057851,0.130159,0.063291,0.215116,0.248101,0.314286,0.038298,0.152918,0.128395,0.232092,0.073298,0.285,0.193460,0.010977,0.001233,0.000002,1.597493e-34
1374,0.2,0.346531,0.358370,0.256410,0.124664,0.310345,0.794118,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.037578,0.224138,0.158717,0.063361,0.053968,0.044304,0.093023,0.086076,0.106122,0.302128,0.476861,0.439506,0.283668,0.170157,0.100,0.629428,0.047201,0.014229,0.000011,2.234494e-50
1574,0.2,0.207415,0.210671,0.169231,0.081614,0.241379,0.843137,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.104384,0.079310,0.074013,0.068871,0.047619,0.063291,0.145349,0.101266,0.159184,0.378723,0.265594,0.113580,0.226361,0.060209,0.250,0.160763,0.031833,0.001233,0.005805,9.257721e-11


In [ ]:
#combined_model = GradientBoostingClassifier()
combined_model = RandomForestClassifier()
combined_model.fit(X_pred_train_meta, y_pred_train)

RandomForestClassifier()

In [ ]:
# scores on test data
###
y_pred = clf1.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  -------------------- RF ------------------------- ")
print("  precision: ", precision_score(y_test, y_pred))
print("  recall: ", recall_score(y_test, y_pred))
print("  f1-score: ", f1_score(y_test, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

###
y_pred = clf2.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  -------------------- GB ------------------------- ")
print("  precision: ", precision_score(y_test, y_pred))
print("  recall: ", recall_score(y_test, y_pred))
print("  f1-score: ", f1_score(y_test, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

###
y_pred = clf3.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  -------------------- NN ------------------------- ")
print("  precision: ", precision_score(y_test, y_pred))
print("  recall: ", recall_score(y_test, y_pred))
print("  f1-score: ", f1_score(y_test, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

###
y_pred = clf2.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  -------------------- NB ------------------------- ")
print("  precision: ", precision_score(y_test, y_pred))
print("  recall: ", recall_score(y_test, y_pred))
print("  f1-score: ", f1_score(y_test, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

  -------------------- RF ------------------------- 
  precision:  0.5
  recall:  0.06666666666666667
  f1-score:  0.11764705882352941
  fpr:  0.003194888178913738
  tn:  312
  fp:  1
  fn:  14
  tp:  1
  real_cheaters:  15
  -------------------- GB ------------------------- 
  precision:  0.7142857142857143
  recall:  0.3333333333333333
  f1-score:  0.4545454545454545
  fpr:  0.006389776357827476
  tn:  311
  fp:  2
  fn:  10
  tp:  5
  real_cheaters:  15
  -------------------- NN ------------------------- 
  precision:  1.0
  recall:  0.13333333333333333
  f1-score:  0.23529411764705882
  fpr:  0.0
  tn:  313
  fp:  0
  fn:  13
  tp:  2
  real_cheaters:  15
  -------------------- NB ------------------------- 
  precision:  0.7142857142857143
  recall:  0.3333333333333333
  f1-score:  0.4545454545454545
  fpr:  0.006389776357827476
  tn:  311
  fp:  2
  fn:  10
  tp:  5
  real_cheaters:  15


In [ ]:
P1_test = clf1.predict_proba(X_test)
P1_test = P1_test[:, 1]
P2_test = clf2.predict_proba(X_test)
P2_test = P2_test[:, 1]
P3_test = clf3.predict_proba(X_test)
P3_test = P3_test[:, 1]
P4_test = clf4.predict_proba(X_test)
P4_test = P4_test[:, 1]

X_test["Predict1"] = P1_test
X_test["Predict2"] = P2_test
X_test["Predict3"] = P3_test
X_test["Predict4"] = P4_test

X_test_meta = X_test.loc[:, col_features]

combined_model_predictions = combined_model.predict(X_test_meta)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, combined_model_predictions).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_test, combined_model_predictions))
print("  recall: ", recall_score(y_test, combined_model_predictions))
print("  f1-score: ", f1_score(y_test, combined_model_predictions))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)


  precision:  0.875
  recall:  0.4666666666666667
  f1-score:  0.608695652173913
  fpr:  0.003194888178913738
  tn:  312
  fp:  1
  fn:  8
  tp:  7
  real_cheaters:  15
